In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('2-fft-malicious-n-0-3-m-1-data.csv')
df2 = pd.read_csv('2-fft-malicious-n-0-5-m-9-data.csv')
df3 = pd.read_csv('2-fft-malicious-n-0-10-m-14-data.csv')
df4 = pd.read_csv('2-fft-malicious-n-0-12-m-9-data.csv')
df5 = pd.read_csv('2-fft-malicious-n-0-15-m-1-data.csv')

In [3]:
df1 = df1.loc[df1['router'] == 1]
df2 = df2.loc[df2['router'] == 1]
df3 = df3.loc[df3['router'] == 1]
df4 = df4.loc[df4['router'] == 1]
df5 = df5.loc[df5['router'] == 1]

In [4]:
df1 = df1.drop(columns=['router'])
df2 = df2.drop(columns=['router'])
df3 = df3.drop(columns=['router'])
df4 = df4.drop(columns=['router'])
df5 = df5.drop(columns=['router'])

In [5]:
df1.to_csv('2-fft-normal-n-0-3-data-r1-mal.csv',index=False)
df2.to_csv('2-fft-normal-n-0-5-data-r1-mal.csv',index=False)
df3.to_csv('2-fft-normal-n-0-10-data-r1-mal.csv',index=False)
df4.to_csv('2-fft-normal-n-0-12-data-r1-mal.csv',index=False)
df5.to_csv('2-fft-normal-n-0-15-data-r1-mal.csv',index=False)

In [6]:
df1 = pd.read_csv('2-fft-normal-n-0-3-data-r1-mal.csv')
df2 = pd.read_csv('2-fft-normal-n-0-5-data-r1-mal.csv')
df3 = pd.read_csv('2-fft-normal-n-0-10-data-r1-mal.csv')
df4 = pd.read_csv('2-fft-normal-n-0-12-data-r1-mal.csv')
df5 = pd.read_csv('2-fft-normal-n-0-15-data-r1-mal.csv')

In [7]:
def timecount(df):
    timearr = []
    interval = 99
    count = 0
    for index, row in df.iterrows():
        if row["timestamp"]<=interval:
            count+=1
        else:
            timearr.append([interval+1,count])
            count=1
            interval+=100
    timearr.append([interval+1,count])
    return timearr

In [8]:
def maxcount(timearr,df):
    countarr = []
    increarr = []
    maxarr = []
    for i in range(len(timearr)):
        for cnt in range(timearr[i][1],0,-1):
            countarr.append(cnt)
            maxarr.append(timearr[i][1])
            increment = timearr[i][1] - cnt + 1
            increarr.append(increment)
    df = df.assign(packet_count_decr=countarr)
    df = df.assign(packet_count_incr=increarr)
    df = df.assign(max_packet_count=maxarr)
    return df

In [9]:
df1 = maxcount(timecount(df1),df1)
df2 = maxcount(timecount(df2),df2)
df3 = maxcount(timecount(df3),df3)
df4 = maxcount(timecount(df4),df4)
df5 = maxcount(timecount(df5),df5)

In [10]:
df1

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,dst_ni,dst_router,traversal_id,hop_count,current_hop,hop_percentage,enqueue_time,packet_count_decr,packet_count_incr,max_packet_count
0,7,2,0,1,1,4,3,9,1,1,23,7,1,3,0,0.00,4,8,1,8
1,11,2,4,1,1,4,3,9,0,0,23,7,0,4,1,0.25,8,7,2,8
2,84,0,1,4,1,1,5,17,23,7,1,1,1,3,3,1.00,16,6,3,8
3,84,4,2,1,1,4,3,9,3,3,24,8,3,5,2,0.40,12,5,4,8
4,85,0,1,4,2,2,5,17,23,7,1,1,1,3,3,1.00,17,4,5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89536,2490410,0,4,5,3,2,3,9,0,0,17,1,20266,1,1,1.00,22,5,3,7
89537,2490417,0,4,5,4,2,3,9,0,0,17,1,20266,1,1,1.00,29,4,4,7
89538,2490424,0,4,5,5,3,3,9,0,0,17,1,20266,1,1,1.00,36,3,5,7
89539,2490425,0,4,1,1,4,3,10,0,0,17,1,20267,1,1,1.00,36,2,6,7


In [11]:
def rename(df):
    df["packet_count_index"] = df["packet_count_decr"]*df["packet_count_incr"]
    df["packet_max_index"] = df["packet_count_index"]*df["max_packet_count"]
    df["port_index"] = df["outport"]*df["inport"]
    df["cache_coherence_flit_index"] = df["cache_coherence_type"]*df["flit_id"]
    df["flit_index"] = df["cache_coherence_flit_index"]*df["flit_type"]
    df["traversal_index"] = df["flit_index"]*df["traversal_id"]
    df["cache_coherence_vnet_index"] = df["cache_coherence_type"]*df["vnet"]
    df["vnet_vc_index"] = df["vnet"]*df["vc"]
    df["vnet_vc_cc_index"] = df["vnet"]*df["cache_coherence_vnet_index"]
    df.head(50)

In [12]:
rename(df1)
rename(df2)
rename(df3)
rename(df4)
rename(df5)

In [13]:
df1['target'] = 0
df2['target'] = 0
df3['target'] = 0
df4['target'] = 0
df5['target'] = 0

In [14]:
df1

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,7,2,0,1,1,4,3,9,1,1,...,8,64,0,1,4,4,3,27,9,0
1,11,2,4,1,1,4,3,9,0,0,...,14,112,8,1,4,0,3,27,9,0
2,84,0,1,4,1,1,5,17,23,7,...,18,144,0,4,4,4,20,85,100,0
3,84,4,2,1,1,4,3,9,3,3,...,20,160,8,1,4,12,3,27,9,0
4,85,0,1,4,2,2,5,17,23,7,...,20,160,0,8,16,16,20,85,100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89536,2490410,0,4,5,3,2,3,9,0,0,...,15,105,0,15,30,607980,15,27,45,0
89537,2490417,0,4,5,4,2,3,9,0,0,...,16,112,0,20,40,810640,15,27,45,0
89538,2490424,0,4,5,5,3,3,9,0,0,...,15,105,0,25,75,1519950,15,27,45,0
89539,2490425,0,4,1,1,4,3,10,0,0,...,12,84,0,1,4,81068,3,30,9,0


In [15]:
df2

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,0,3,27,9,0
1,240,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,32,3,27,9,0
2,359,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,44,3,27,9,0
3,458,1,4,1,1,4,3,9,0,0,...,1,1,4,1,4,56,3,27,9,0
4,475,0,1,1,1,4,3,9,5,5,...,2,4,0,1,4,60,3,27,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43464,2479828,4,2,6,2,2,3,9,18,2,...,10,60,8,12,24,489360,18,27,54,0
43465,2479835,4,2,6,3,2,3,9,18,2,...,12,72,8,18,36,734040,18,27,54,0
43466,2479842,4,2,6,4,2,3,9,18,2,...,12,72,8,24,48,978720,18,27,54,0
43467,2479849,4,2,6,5,3,3,9,18,2,...,10,60,8,30,90,1835100,18,27,54,0


In [16]:
df3

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,0,3,27,9,0
1,240,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,32,3,27,9,0
2,359,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,44,3,27,9,0
3,458,1,4,1,1,4,3,9,0,0,...,1,1,4,1,4,56,3,27,9,0
4,483,0,1,1,1,4,3,9,10,10,...,2,4,0,1,4,60,3,27,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40772,2452347,4,2,6,5,3,3,9,18,2,...,1,1,8,30,90,1740420,18,27,54,0
40773,2452348,4,2,2,1,4,3,10,18,2,...,1,1,8,2,8,154712,6,30,18,0
40774,2452349,4,2,7,1,4,3,11,18,2,...,1,1,8,7,28,541464,21,33,63,0
40775,2452581,2,4,5,1,1,3,9,0,0,...,1,1,8,5,5,96740,15,27,45,0


In [17]:
df4

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,0,3,27,9,0
1,222,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,28,3,27,9,0
2,340,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,40,3,27,9,0
3,438,1,4,1,1,4,3,9,0,0,...,1,1,4,1,4,52,3,27,9,0
4,497,0,1,1,1,4,3,9,9,9,...,1,1,0,1,4,60,3,27,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44084,2509820,2,4,5,2,2,3,9,0,0,...,5,25,8,10,20,419140,15,27,45,0
44085,2509827,2,4,5,3,2,3,9,0,0,...,8,40,8,15,30,628710,15,27,45,0
44086,2509834,2,4,5,4,2,3,9,0,0,...,9,45,8,20,40,838280,15,27,45,0
44087,2509841,2,4,5,5,3,3,9,0,0,...,8,40,8,25,75,1571775,15,27,45,0


In [18]:
df5

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,7,2,0,1,1,4,3,9,1,1,...,7,49,0,1,4,4,3,27,9,0
1,11,2,4,1,1,4,3,9,0,0,...,12,84,8,1,4,0,3,27,9,0
2,88,0,1,4,1,1,5,17,23,7,...,15,105,0,4,4,4,20,85,100,0
3,89,0,1,4,2,2,5,17,23,7,...,16,112,0,8,16,16,20,85,100,0
4,90,0,1,4,3,2,5,17,23,7,...,15,105,0,12,24,24,20,85,100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82027,2492827,0,4,1,1,4,3,10,0,0,...,25,225,0,1,4,81248,3,30,9,0
82028,2492834,4,0,6,1,1,3,9,17,1,...,24,216,0,6,6,121866,18,27,54,0
82029,2492841,4,0,6,2,2,3,9,17,1,...,21,189,0,12,24,487464,18,27,54,0
82030,2492848,4,0,6,3,2,3,9,17,1,...,16,144,0,18,36,731196,18,27,54,0


In [19]:
df1.dtypes

timestamp                       int64
outport                         int64
inport                          int64
cache_coherence_type            int64
flit_id                         int64
flit_type                       int64
vnet                            int64
vc                              int64
src_ni                          int64
src_router                      int64
dst_ni                          int64
dst_router                      int64
traversal_id                    int64
hop_count                       int64
current_hop                     int64
hop_percentage                float64
enqueue_time                    int64
packet_count_decr               int64
packet_count_incr               int64
max_packet_count                int64
packet_count_index              int64
packet_max_index                int64
port_index                      int64
cache_coherence_flit_index      int64
flit_index                      int64
traversal_index                 int64
cache_cohere

In [20]:
df1.to_csv('2-fft-normal-n-0-3-data-r1-mal.csv',index=False)
df2.to_csv('2-fft-normal-n-0-5-data-r1-mal.csv',index=False)
df3.to_csv('2-fft-normal-n-0-10-data-r1-mal.csv',index=False)
df4.to_csv('2-fft-normal-n-0-12-data-r1-mal.csv',index=False)
df5.to_csv('2-fft-normal-n-0-15-data-r1-mal.csv',index=False)